## First star

In [1]:
import sys
import copy
import re

In [2]:
sys.path.append("../")

In [3]:
file_path = 'input_ag.txt'

In [4]:
starting_series = []
with open(file_path, 'r') as file:
    for line in file:
        starting_series.append(line.strip('\n'))

In [16]:
def solve_puzzle(grid):
    # grid is a list of strings
    rows = len(grid)
    cols = len(grid[0]) if rows > 0 else 0
    
    visited = [[False]*cols for _ in range(rows)]

    def neighbors(r, c):
        # Return valid orthogonal neighbors
        for nr, nc in [(r-1,c),(r+1,c),(r,c-1),(r,c+1)]:
            if 0 <= nr < rows and 0 <= nc < cols:
                yield nr, nc

    def dfs(r, c):
        # Perform DFS from (r,c) to find all connected plots of the same type
        region_type = grid[r][c]
        stack = [(r, c)]
        region_cells = []
        visited[r][c] = True
        
        while stack:
            cr, cc = stack.pop()
            region_cells.append((cr, cc))
            for nr, nc in neighbors(cr, cc):
                if not visited[nr][nc] and grid[nr][nc] == region_type:
                    visited[nr][nc] = True
                    stack.append((nr, nc))
        
        return region_cells

    total_price = 0

    for r in range(rows):
        for c in range(cols):
            if not visited[r][c]:
                # Find a new region
                region_cells = dfs(r, c)
                area = len(region_cells)
                
                # Calculate perimeter
                # For each cell in the region, count how many of its 4 sides either go out of the grid
                # or border a different type.
                perimeter = 0
                region_type = grid[region_cells[0][0]][region_cells[0][1]]
                for (cr, cc) in region_cells:
                    # Check all 4 sides
                    # Up
                    if cr == 0 or grid[cr-1][cc] != region_type:
                        perimeter += 1
                    # Down
                    if cr == rows-1 or grid[cr+1][cc] != region_type:
                        perimeter += 1
                    # Left
                    if cc == 0 or grid[cr][cc-1] != region_type:
                        perimeter += 1
                    # Right
                    if cc == cols-1 or grid[cr][cc+1] != region_type:
                        perimeter += 1

                price = area * perimeter
                total_price += price

    return total_price


# Example usage:
if __name__ == "__main__":
    # Example 1 from the problem statement
    example1 = [
        "AAAA",
        "BBCD",
        "BBCC",
        "EEEC"
    ]
    print(solve_puzzle(example1))  # Expected total: 140

    # Example 2 (O and X example)
    example2 = [
        "OOOOO",
        "OXOXO",
        "OOOOO",
        "OXOXO",
        "OOOOO"
    ]
    print(solve_puzzle(example2))  # Expected total: 772

    # Larger example
    example3 = [
        "RRRRIICCFF",
        "RRRRIICCCF",
        "VVRRRCCFFF",
        "VVRCCCJFFF",
        "VVVVCJJCFE",
        "VVIVCCJJEE",
        "VVIIICJJEE",
        "MIIIIIJJEE",
        "MIIISIJEEE",
        "MMMISSJEEE"
    ]
    print(solve_puzzle(example3))  # Expected total: 1930

140
772
1930


In [8]:
print(solve_puzzle(starting_series)) 

1370258


## Second star

In [31]:
def compute_total_price(grid):
    """
    Computes the total price of fencing all regions on the map based on the new bulk discount method.

    Parameters:
        grid (list of str): The grid representing different plant types.

    Returns:
        int: The total fencing price.
    """
    rows = len(grid)
    cols = len(grid[0]) if rows > 0 else 0

    visited = [[False] * cols for _ in range(rows)]

    def neighbors(r, c):
        # Yield valid orthogonal neighbors
        for nr, nc in [(r - 1, c), (r + 1, c), (r, c - 1), (r, c + 1)]:
            if 0 <= nr < rows and 0 <= nc < cols:
                yield nr, nc

    def dfs(r, c):
        """
        Performs DFS to find all cells in the same region.

        Returns:
            list of tuple: List of (row, column) tuples in the region.
        """
        stack = [(r, c)]
        region_cells = []
        visited[r][c] = True
        region_type = grid[r][c]

        while stack:
            cr, cc = stack.pop()
            region_cells.append((cr, cc))
            for nr, nc in neighbors(cr, cc):
                if not visited[nr][nc] and grid[nr][nc] == region_type:
                    visited[nr][nc] = True
                    stack.append((nr, nc))

        return region_cells

    def count_sides(region_cells):
        """
        Counts the number of straight fence sides for a given region.

        Parameters:
            region_cells (list of tuple): List of (row, column) tuples in the region.

        Returns:
            int: Number of sides for the region.
        """
        # Initialize boundary edge matrices
        horizontal_boundaries = [[False] * cols for _ in range(rows + 1)]  # horizontal_boundaries[r][c] is between (r-1,c) and (r,c)
        vertical_boundaries = [[False] * (cols + 1) for _ in range(rows)]    # vertical_boundaries[r][c] is between (r,c-1) and (r,c)

        region_type = grid[region_cells[0][0]][region_cells[0][1]]

        for r, c in region_cells:
            # Check Up
            if r == 0 or grid[r - 1][c] != region_type:
                horizontal_boundaries[r][c] = True
            # Check Down
            if r == rows - 1 or grid[r + 1][c] != region_type:
                horizontal_boundaries[r + 1][c] = True
            # Check Left
            if c == 0 or grid[r][c - 1] != region_type:
                vertical_boundaries[r][c] = True
            # Check Right
            if c == cols - 1 or grid[r][c + 1] != region_type:
                vertical_boundaries[r][c + 1] = True

        sides = 0

        # Count horizontal sides
        for r in range(rows + 1):
            c = 0
            while c < cols:
                if horizontal_boundaries[r][c]:
                    # Start of a new horizontal side
                    sides += 1
                    # Move through contiguous horizontal boundaries
                    while c < cols and horizontal_boundaries[r][c]:
                        c += 1
                else:
                    c += 1

        # Count vertical sides
        for c in range(cols + 1):
            r = 0
            while r < rows:
                if vertical_boundaries[r][c]:
                    # Start of a new vertical side
                    sides += 1
                    # Move through contiguous vertical boundaries
                    while r < rows and vertical_boundaries[r][c]:
                        r += 1
                else:
                    r += 1

        return sides

    total_price = 0

    for r in range(rows):
        for c in range(cols):
            if not visited[r][c]:
                # Find a new region
                region_cells = dfs(r, c)
                print(region_cells)
                area = len(region_cells)
                sides = count_sides(region_cells)
                price = area * sides
                total_price += price
                print(f"price of the region {price}")
    return total_price

# Example usage and test cases
if __name__ == "__main__":
    # Example 1 from the problem statement
    example1 = [
        "AAAA",
        "BBCD",
        "BBCC",
        "EEEC"
    ]
    print(compute_total_price(example1))  # Expected total: 80

    # Example 2 (O and X example)
    example2 = [
        "OOOOO",
        "OXOXO",
        "OOOOO",
        "OXOXO",
        "OOOOO"
    ]
    print(compute_total_price(example2))  # Expected total: 436

    # E-shaped region example
    example3 = [
        "EEEEE",
        "EXXXX",
        "EEEEE",
        "EXXXX",
        "EEEEE"
    ]
    print(compute_total_price(example3))  # Expected total: 236

    # Another map with complex regions
    example4 = [
        "AAAAAA",
        "AAABBA",
        "AAABBA",
        "ABBAAA",
        "ABBAAA",
        "AAAAAA"
    ]
    print(compute_total_price(example4))  # Expected total: 368

    # Larger example provided in the problem statement
    example5 = [
        "RRRRIICCFF",
        "RRRRIICCCF",
        "VVRRRCCFFF",
        "VVRCCCJFFF",
        "VVVVCJJCFE",
        "VVIVCCJJEE",
        "VVIIICJJEE",
        "MIIIIIJJEE",
        "MIIISIJEEE",
        "MMMISSJEEE"
    ]
    print(compute_total_price(example5))  # Expected total: 1206


[(0, 0), (0, 1), (0, 2), (0, 3)]
price of the region 16
[(1, 0), (1, 1), (2, 1), (2, 0)]
price of the region 16
[(1, 2), (2, 2), (2, 3), (3, 3)]
price of the region 32
[(1, 3)]
price of the region 4
[(3, 0), (3, 1), (3, 2)]
price of the region 12
80
[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (1, 4), (2, 4), (2, 3), (2, 2), (2, 1), (2, 0), (3, 0), (4, 0), (4, 1), (4, 2), (4, 3), (4, 4), (3, 2), (3, 4), (1, 2), (1, 0)]
price of the region 420
[(1, 1)]
price of the region 4
[(1, 3)]
price of the region 4
[(3, 1)]
price of the region 4
[(3, 3)]
price of the region 4
436
[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (1, 0), (2, 0), (2, 1), (2, 2), (2, 3), (2, 4), (3, 0), (4, 0), (4, 1), (4, 2), (4, 3), (4, 4)]
price of the region 204
[(1, 1), (1, 2), (1, 3), (1, 4)]
price of the region 16
[(3, 1), (3, 2), (3, 3), (3, 4)]
price of the region 16
236
[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (1, 5), (2, 5), (3, 5), (3, 4), (3, 3), (4, 3), (5, 3), (5, 4), (5, 5), (5, 2), (5, 1), (5, 0), (4, 0),

In [33]:
region_test = [(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (1, 5), (2, 5), (3, 5), (3, 4), (3, 3), (4, 3), (5, 3), (5, 4), (5, 5), (5, 2), (5, 1), (5, 0), (4, 0), (3, 0), (2, 0), (2, 1), (2, 2), (4, 4), (4, 5), (1, 2), (1, 1), (1, 0)]
len(region_test)

28